<a href="https://colab.research.google.com/github/seungyeon38/Webtoon_recommendation_system/blob/master/CountVectorize%20%2B%20TFIDFVectorize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
import numpy as np
import pandas as pd
import string
string.punctuation
from math import log
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
import re
from tqdm import tqdm
from gensim.summarization import keywords
from textblob import TextBlob

!pip install gensim
!pip install konlpy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [96]:
comic = pd.read_csv('/content/drive/MyDrive/webtoon/Webtoon Dataset.csv')

In [97]:
input_comic = input()

True Beauty


In [98]:
 selected_comic = comic.loc[(comic.Name == input_comic)]

 print(selected_comic['Name'])

1    True Beauty
Name: Name, dtype: object


In [99]:
# 축약형 

contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", "she's": "she is"}

def regularize_contractions(text):

    contractionfree_list = []
    
    for word in text.split(" "):
      if word in contractions.keys():
        contractionfree_list.append(contractions[word])
      else :
        contractionfree_list.append(word)
      
    return " ".join(contractionfree_list)


In [100]:
# defining the function to remove punctuation
# 글자가 puctuation에 해당하면 " "를, 해당하지 않으면 글자 그대로

def remove_punctuation(text):
    punctuationfree_list = []

    for word in text:
      if word in string.punctuation:
        punctuationfree_list.append(" ")
      else :
        punctuationfree_list.append(word)
    return "".join(punctuationfree_list)

In [101]:
# defining function for tokenization
# 영어만 단어단위로 토큰화

p = re.compile('[a-z]+')

def tokenization(text):
    # print(text)
    result = p.findall(text)
    # print(result)
    return result

In [102]:
# stop words present in the library

stopwords = nltk.corpus.stopwords.words('english')

# defining the function to remove stopwords from tokenized text
# stopwords 제거 

def remove_stopwords(text):
    for word in text:
      output= [word for word in text if word not in stopwords]
    return output

In [103]:
# setting lower case

comic['Lower_Summary'] = comic['Summary'].apply(lambda x: x.lower())
comic['Lower_Summary'].head(3)

0    she's young, single and about to achieve her d...
1    after binge-watching beauty videos online, a s...
2    after making a grisly discovery in the country...
Name: Lower_Summary, dtype: object

In [104]:
# setting no contraction form

comic['No_Contraction_Summary'] = comic['Lower_Summary'].apply(lambda x:regularize_contractions(x))
comic['No_Contraction_Summary'].head(3)

0    she is young, single and about to achieve her ...
1    after binge-watching beauty videos online, a s...
2    after making a grisly discovery in the country...
Name: No_Contraction_Summary, dtype: object

In [105]:
# setting punctuation removed case

comic['Clean_Summary'] = comic['No_Contraction_Summary'].apply(lambda x:remove_punctuation(x))
comic['Clean_Summary'].head(3)

0    she is young  single and about to achieve her ...
1    after binge watching beauty videos online  a s...
2    after making a grisly discovery in the country...
Name: Clean_Summary, dtype: object

In [106]:
# Tokenize column

comic['Tokenized_Summary'] = comic['Clean_Summary'].apply(lambda x: tokenization(x))
comic['Tokenized_Summary'].head(3)

0    [she, is, young, single, and, about, to, achie...
1    [after, binge, watching, beauty, videos, onlin...
2    [after, making, a, grisly, discovery, in, the,...
Name: Tokenized_Summary, dtype: object

In [107]:
# remove Stopwords

comic['No_Stopwords_Summary'] = comic['Tokenized_Summary'].apply(lambda x:remove_stopwords(x))
comic['No_Stopwords_Summary'].head(3)

0    [young, single, achieve, dream, creating, incr...
1    [binge, watching, beauty, videos, online, shy,...
2    [making, grisly, discovery, countryside, small...
Name: No_Stopwords_Summary, dtype: object

In [108]:
# remove verbs

def extract_no_verb(text):
  part_of_speech_text = pos_tag(text)
  result = []
  for word in part_of_speech_text:
    if word[1] != "VB" and word[1] != "VBD" and word[1] != "VBG" and word[1] != "VBN" and word[1] != "VBP" and word[1] != "VBZ":
        result.append(word[0])
  return result; 

comic['No_Verb_Summary'] = comic['No_Stopwords_Summary'].apply(lambda x:extract_no_verb(x))
comic['No_Verb_Summary'].head(3)

0    [young, single, achieve, dream, incredible, vi...
1    [binge, beauty, videos, online, shy, comic, bo...
2    [grisly, discovery, countryside, small, town, ...
Name: No_Verb_Summary, dtype: object

In [109]:
# make words to string

def wordlist_to_string(words_list) :
  doc = []
  for doc_word in words_list :
    doc.append(doc_word)
  return " ".join(doc)

comic['Summary_final'] = comic['No_Verb_Summary'].apply(lambda x:wordlist_to_string(x))
comic['Summary_final'].head(3)

0    young single achieve dream incredible videogam...
1    binge beauty videos online shy comic book fan ...
2    grisly discovery countryside small town book e...
Name: Summary_final, dtype: object

In [110]:
# extract keywords using gensim

def extract_keywords(doc):
  keyword_list = []
  keyword_string = keywords(doc)
  keyword_list = keyword_string.split('\n')

  return keyword_list;

comic['Summary_Keywords'] = comic['Summary_final'].apply(lambda x:extract_keywords(x))

In [111]:
# extract 3 important keywords

def keyword_3(keyword_list):
  keyword_list_3 = keyword_list[0:3]

  return keyword_list_3

comic['Summary_Keywords_3'] = comic['Summary_Keywords'].apply(lambda x:keyword_3(x))
comic.head(5)

,id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link,Lower_Summary,No_Contraction_Summary,Clean_Summary,Tokenized_Summary,No_Stopwords_Summary,No_Verb_Summary,Summary_final,Summary_Keywords,Summary_Keywords_3
0,0,Let's Play,Leeanne M. Krecic (Mongie),30.6M,Romance,9.62,4.2M,"She's young, single and about to achieve her d...",UP EVERY TUESDAY,https://www.webtoons.com/en/romance/letsplay/l...,"she's young, single and about to achieve her d...","she is young, single and about to achieve her ...",she is young single and about to achieve her ...,"[she, is, young, single, and, about, to, achie...","[young, single, achieve, dream, creating, incr...","[young, single, achieve, dream, incredible, vi...",young single achieve dream incredible videogam...,"[popular, videogames, single, troublesome, stay]","[popular, videogames, single]"
1,1,True Beauty,Yaongyi,39.9M,Romance,9.60,6.4M,"After binge-watching beauty videos online, a s...",UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/truebeauty...,"after binge-watching beauty videos online, a s...","after binge-watching beauty videos online, a s...",after binge watching beauty videos online a s...,"[after, binge, watching, beauty, videos, onlin...","[binge, watching, beauty, videos, online, shy,...","[binge, beauty, videos, online, shy, comic, bo...",binge beauty videos online shy comic book fan ...,"[secret, status short, beauty videos]","[secret, status short, beauty videos]"
2,2,Midnight Poppy Land,Lilydusk,10.4M,Romance,9.81,2.1M,After making a grisly discovery in the country...,UP EVERY SATURDAY,https://www.webtoons.com/en/romance/midnight-p...,after making a grisly discovery in the country...,after making a grisly discovery in the country...,after making a grisly discovery in the country...,"[after, making, a, grisly, discovery, in, the,...","[making, grisly, discovery, countryside, small...","[grisly, discovery, countryside, small, town, ...",grisly discovery countryside small town book e...,[deeper dangerous underworld],[deeper dangerous underworld]
3,3,Age Matters,Enjelicious,25.9M,Romance,9.79,3.5M,She's a hopeless romantic who's turning 30's ...,UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/age-matter...,she's a hopeless romantic who's turning 30's ...,she is a hopeless romantic who is turning 30's...,she is a hopeless romantic who is turning 30 s...,"[she, is, a, hopeless, romantic, who, is, turn...","[hopeless, romantic, turning, super, happy, re...","[hopeless, romantic, super, happy, reclusive, ...",hopeless romantic super happy reclusive billio...,"[happy, rules]","[happy, rules]"
4,4,Unholy Blood,Lina Im / Jeonghyeon Kim,9.9M,Supernatural,9.85,1.5M,When vampires destroy her chance to have the n...,UP EVERY THURSDAY,https://www.webtoons.com/en/supernatural/unhol...,when vampires destroy her chance to have the n...,when vampires destroy her chance to have the n...,when vampires destroy her chance to have the n...,"[when, vampires, destroy, her, chance, to, hav...","[vampires, destroy, chance, normal, life, alwa...","[vampires, chance, normal, life, always, hayan...",vampires chance normal life always hayan draw ...,"[hayan, chance normal, ones]","[hayan, chance normal, ones]"


In [112]:
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
from textblob import Word

# find and add synonym of keywords

def keyword_synsets(keywords):
  keyword_synsets = []
  for i in range(len(keywords)):
    keyword_synsets.append(keywords[i])
    word = Word(keywords[i])
    for synset in word.synsets:
      for lemma in synset.lemmas():
        if lemma.name() not in keyword_synsets:
          keyword_synsets.append(lemma.name())

  return keyword_synsets

comic['Summary_Keywords_Synsets'] = comic['Summary_Keywords'].apply(lambda x:keyword_synsets(x))
comic['Summary_Keywords_Synsets'].head(1)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    [popular, democratic, pop, videogames, single,...
Name: Summary_Keywords_Synsets, dtype: object

In [113]:
comic_df = comic[['id','Name', 'Writer', 'Genre', 'Rating', 'Clean_Summary', 'Summary_Keywords',
                 'Summary_Keywords_3', 'Summary_Keywords_Synsets']]
comic_df.head(1)

,id,Name,Writer,Genre,Rating,Clean_Summary,Summary_Keywords,Summary_Keywords_3,Summary_Keywords_Synsets
0,0,Let's Play,Leeanne M. Krecic (Mongie),Romance,9.62,she is young single and about to achieve her ...,"[popular, videogames, single, troublesome, stay]","[popular, videogames, single]","[popular, democratic, pop, videogames, single,..."


In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TfidfVectorize using scikit learn

comic_df['Summary_literal'] = comic_df['Summary_Keywords_Synsets'].apply(lambda x : (' ').join(x))
tfidf_vect = TfidfVectorizer()
tfidf_matrix = tfidf_vect.fit_transform(comic_df['Summary_literal'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [115]:
# evaluate cosine similarity

cosine_sim_tfidf = cosine_similarity(tfidf_matrix, tfidf_matrix)

# make table of webtoon's name and index

indices = pd.Series(comic_df.index, index=comic_df['Name']).drop_duplicates()
print(indices.head())

Name
Let's Play             0
True Beauty            1
Midnight Poppy Land    2
Age Matters            3
Unholy Blood           4
dtype: int64


In [116]:
# webtoon recommendation fuction by TfidfVectorize

def comic_tfidf_rec(title, cosine_sim=cosine_sim_tfidf):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    comic_indices = [i[0] for i in sim_scores]
    
    result_df = comic_df.iloc[comic_indices].copy()
    result_df['sim_score'] = [i[1] for i in sim_scores]

    return result_df

comic_tfidf_rec(input_comic)

,id,Name,Writer,Genre,Rating,Clean_Summary,Summary_Keywords,Summary_Keywords_3,Summary_Keywords_Synsets,Summary_literal,sim_score
469,469,My Deepest Secret,Hanza Art,Thriller,9.68,having a handsome kind and caring boyfriend l...,"[secret, kind]","[secret, kind]","[secret, arcanum, mystery, enigma, closed_book...",secret arcanum mystery enigma closed_book clan...,0.850389
270,270,Mosquito Wars,JH,Sci-fi,9.62,one hundred years after mosquitoes have evolve...,"[humans, humanity, secret, team]","[humans, humanity, secret]","[humans, world, human_race, humanity, humankin...",humans world human_race humanity humankind hum...,0.788014
65,65,CODE HELIX,Na-eum So,Drama,9.36,rendra an intelligent high school student kno...,"[classmate, classmates helix, rendra, discover...","[classmate, classmates helix, rendra]","[classmate, schoolmate, schoolfellow, class_fe...",classmate schoolmate schoolfellow class_fellow...,0.586609
16,16,Sixth Sense Kiss,Got W / Jocobong,Romance,9.71,yesul hong has a secret when she kisses som...,"[secret, work, man, accidentally, vision]","[secret, work, man]","[secret, arcanum, mystery, enigma, closed_book...",secret arcanum mystery enigma closed_book clan...,0.492236
28,28,Secret Playlist,2F,Romance,9.66,hanju is your ordinary university studentâ€¦ e...,"[cover, levi, plii, secret]","[cover, levi, plii]","[cover, screen, covert, concealment, blanket, ...",cover screen covert concealment blanket coveri...,0.479251
49,49,The Girl Downstairs,Songah Min,Romance,9.60,when joon moves into his new apartment on his ...,"[joon, mysterious]","[joon, mysterious]","[joon, mysterious, cryptic, cryptical, deep, i...",joon mysterious cryptic cryptical deep inscrut...,0.236159
196,196,The Newlywed Diaryof a Witch and a Dragon,New lung / Dimang,Romance,9.28,after spending the night with a recent hire of...,"[aiden, wendy, secrets, night recent]","[aiden, wendy, secrets]","[aiden, wendy, secrets, secret, arcanum, myste...",aiden wendy secrets secret arcanum mystery eni...,0.211406
524,524,Uriah,Toffuo,Horror,9.80,a rainy night a mysterious island a boy with...,"[story, secrets, journey]","[story, secrets, journey]","[story, narrative, narration, tale, floor, lev...",story narrative narration tale floor level sto...,0.146800
374,374,Girls Have a Blog,Sarah Bollinger / Tara Kurtzhals,Slice of life,9.17,creating comics playing videos games eating ...,[videos games],[videos games],[videos games],videos games,0.137256
476,476,Flawless,Shinshinhye,Romance,9.68,sarah is a tough girl who prides herself for b...,"[mystery, prides, street]","[mystery, prides, street]","[mystery, enigma, secret, closed_book, mystery...",mystery enigma secret closed_book mystery_stor...,0.133837


In [117]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorize using scikit learn

comic_df['Summary_literal'] = comic_df['Summary_Keywords_Synsets'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
count_matrix = count_vect.fit_transform(comic_df['Summary_literal'])
print(count_matrix.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


(569, 14629)


In [118]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim_count = cosine_similarity(count_matrix, count_matrix)

indices = pd.Series(comic_df.index, index=comic_df['Name']).drop_duplicates()
print(indices.head())

Name
Let's Play             0
True Beauty            1
Midnight Poppy Land    2
Age Matters            3
Unholy Blood           4
dtype: int64


In [119]:
# webtoon recommendation fuction by CountVectorize

def comic_count_REC(title, cosine_sim=cosine_sim_count):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    comic_indices = [i[0] for i in sim_scores]
    
    result_df = comic_df.iloc[comic_indices].copy()
    result_df['sim_score'] = [i[1] for i in sim_scores]

    return result_df

comic_count_REC(input_comic)

,id,Name,Writer,Genre,Rating,Clean_Summary,Summary_Keywords,Summary_Keywords_3,Summary_Keywords_Synsets,Summary_literal,sim_score
469,469,My Deepest Secret,Hanza Art,Thriller,9.68,having a handsome kind and caring boyfriend l...,"[secret, kind]","[secret, kind]","[secret, arcanum, mystery, enigma, closed_book...",secret arcanum mystery enigma closed_book clan...,0.863338
270,270,Mosquito Wars,JH,Sci-fi,9.62,one hundred years after mosquitoes have evolve...,"[humans, humanity, secret, team]","[humans, humanity, secret]","[humans, world, human_race, humanity, humankin...",humans world human_race humanity humankind hum...,0.751439
65,65,CODE HELIX,Na-eum So,Drama,9.36,rendra an intelligent high school student kno...,"[classmate, classmates helix, rendra, discover...","[classmate, classmates helix, rendra]","[classmate, schoolmate, schoolfellow, class_fe...",classmate schoolmate schoolfellow class_fellow...,0.600545
28,28,Secret Playlist,2F,Romance,9.66,hanju is your ordinary university studentâ€¦ e...,"[cover, levi, plii, secret]","[cover, levi, plii]","[cover, screen, covert, concealment, blanket, ...",cover screen covert concealment blanket coveri...,0.543852
16,16,Sixth Sense Kiss,Got W / Jocobong,Romance,9.71,yesul hong has a secret when she kisses som...,"[secret, work, man, accidentally, vision]","[secret, work, man]","[secret, arcanum, mystery, enigma, closed_book...",secret arcanum mystery enigma closed_book clan...,0.502939
196,196,The Newlywed Diaryof a Witch and a Dragon,New lung / Dimang,Romance,9.28,after spending the night with a recent hire of...,"[aiden, wendy, secrets, night recent]","[aiden, wendy, secrets]","[aiden, wendy, secrets, secret, arcanum, myste...",aiden wendy secrets secret arcanum mystery eni...,0.245040
49,49,The Girl Downstairs,Songah Min,Romance,9.60,when joon moves into his new apartment on his ...,"[joon, mysterious]","[joon, mysterious]","[joon, mysterious, cryptic, cryptical, deep, i...",joon mysterious cryptic cryptical deep inscrut...,0.197969
524,524,Uriah,Toffuo,Horror,9.80,a rainy night a mysterious island a boy with...,"[story, secrets, journey]","[story, secrets, journey]","[story, narrative, narration, tale, floor, lev...",story narrative narration tale floor level sto...,0.152586
476,476,Flawless,Shinshinhye,Romance,9.68,sarah is a tough girl who prides herself for b...,"[mystery, prides, street]","[mystery, prides, street]","[mystery, enigma, secret, closed_book, mystery...",mystery enigma secret closed_book mystery_stor...,0.118678
222,222,The Lazy Lord Masters the Sword,Dodomoon / doip,Fantasy,9.65,after witnessing the shocking death of his mot...,"[whispers intention, complete, mysterious]","[whispers intention, complete, mysterious]","[whispers intention, complete, finish, dispatc...",whispers intention complete finish dispatch di...,0.105492
